[View in Colaboratory](https://colab.research.google.com/github/gmihaila/deep_learning_toy_models/blob/master/keras_checkpoins.ipynb)

### Keras checkpoints

by GeorgeM

In [1]:
# Downlod dataset

!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

!ls

--2018-04-23 15:02:20--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.01s   

2018-04-23 15:02:20 (1.60 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]

datalab  pima-indians-diabetes.data.csv


In [7]:
ls

data.h5   pima-indians-diabetes.data.csv
datalab/  pima-indians-diabetes.data.csv.1


In [2]:
# Checkpoint the weights for best model on validation accuracy
from keras.models import Sequential
from keras.layers import Dense
from keras import callbacks
import matplotlib.pyplot as plt
import numpy
import time

Using TensorFlow backend.


In [0]:
# Import data

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [0]:
# create model

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# checkpoint
filepath="epoch_{epoch:02d}_acc_{val_loss:.4f}.hdf5"
best_model = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
# filepath : path and name of file to save.
# verbose : 0 won't print 1 will print on each epoch if improved or not.
# save_best_only : the latest best model according to the quantity monitored will not be overwritten.
# save_weights_only : True - only model weights saved. False - whole model saved model.save(filepath).
# mode : maximization or the minimization of the monitored quantity e.g. val_loss mode=min
# period : how often to cheack. e.g. period=2 ever 2 epoch will check


early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='min')
# monitor : can be val_loss or val_acc. We want to decrease val_loss and increase val_acc
# min_delta : the minimum value that count as improvement e.g. delta = current_val - previous_val
# patience : how many times we allow the model to NOT improve e.g. patience=2 at the second time the model did not improve will STOP
# verbose : will print a message of the epoch where it early stopped
# mode : which direction we want to go max, min or auto e.g. val_loss we use mode=min val_acc we use mode=max


# When NaN loss is encounter, it will stop
nan_terminate = callbacks.TerminateOnNaN()

# Use tensor board
tensor_board = callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [best_model, nan_terminate, early_stop, tensor_board]


In [7]:
# Fit the model
s = time.time()
model.fit(X, Y, validation_split=0.33, epochs=5, batch_size=2, callbacks=callbacks_list, verbose=1)
# model.fit(X, Y, validation_split=0.33, epochs=5, batch_size=2, verbose=0)

e = time.time()
print '\n\nTIME: ',(e-s)

Train on 514 samples, validate on 254 samples
Epoch 1/5
514/514 [==============================] - 1s 2ms/step - loss: 0.6810 - acc: 0.6420 - val_loss: 0.6619 - val_acc: 0.6732

Epoch 00001: val_loss improved from inf to 0.66191, saving model to epoch_01_acc_0.6619.hdf5
Epoch 2/5
514/514 [==============================] - 1s 1ms/step - loss: 0.6597 - acc: 0.6401 - val_loss: 0.6535 - val_acc: 0.6732

Epoch 00002: val_loss improved from 0.66191 to 0.65349, saving model to epoch_02_acc_0.6535.hdf5
Epoch 3/5
514/514 [==============================] - 1s 1ms/step - loss: 0.6510 - acc: 0.6401 - val_loss: 0.6276 - val_acc: 0.6732

Epoch 00003: val_loss improved from 0.65349 to 0.62758, saving model to epoch_03_acc_0.6276.hdf5
Epoch 4/5
514/514 [==============================] - 1s 991us/step - loss: 0.6382 - acc: 0.6518 - val_loss: 0.6220 - val_acc: 0.7323

Epoch 00004: val_loss improved from 0.62758 to 0.62203, saving model to epoch_04_acc_0.6220.hdf5
Epoch 5/5
514/514 [=====================

In [0]:
rm ./*.hdf5

In [10]:
ls

datalab/                  epoch_05_acc_0.6182.hdf5
epoch_01_acc_0.6536.hdf5  Graph/
epoch_03_acc_0.6351.hdf5  pima-indians-diabetes.data.csv
epoch_04_acc_0.6316.hdf5


In [11]:
!tensorboard --logdir ./Graph 

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.7.0 at http://54bb71543568:6006 (Press CTRL+C to quit)
^C
